## Chatbot

### Set up Telegram and Dialogflow

Library used 
- telegram 
- dialogflow 
- request 
- regex 
- beautiful soup
- pandas 
- numpy
- yahoo finance
- ta 
- plotly 




In [1]:
API_KEY = '5122878572:AAE2oGn3R3A0elWOaGymQsQTj4j2npWaHno'

In [2]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import requests
import re
import os
from google.cloud import dialogflow
import json

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\Zac\Downloads\stock-recommendator-otox-54b9d5ee241b.json"

In [4]:
def detect_intent_texts(project_id, session_id, text, language_code):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversation."""

    session_client = dialogflow.SessionsClient()

    session = session_client.session_path(project_id, session_id)
    print("Session path: {}\n".format(session))

    text_input = dialogflow.TextInput(text=text, language_code=language_code)
    query_input = dialogflow.QueryInput(text=text_input)

    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )
    
    return response

### Performance Sector

In [5]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import yfinance as yf

In [6]:
def sector_classification(url):   
    response = requests.get(url, headers={'User-Agent':'test'})
    soup = BeautifulSoup(response.content, "html.parser")
    soup_list = soup.find(id = "MainContent_tStock")
    sector_stock = [stock.getText() for stock in soup_list.find_all('a')]
    
    stock_list = []
    for stock in sector_stock:
        refined = re.findall(r'[0-9A-Z&]+', stock)
        stock_list.append(refined)     
    return sum(stock_list,[]) 


def code_or_name(stock_list, condition):
    new_stock = []
    if condition == 'code':
        for stock in stock_list:
            if re.findall(r'[0-9]{4}', stock):
                new_stock.append(stock)                
    elif condition == 'name':
        for stock in stock_list:
            if re.findall(r'[A-Z&]+', stock):
                new_stock.append(stock)
    else:
        pass
    return new_stock



def get_market_cap(stock_code): 
    url = "https://www.malaysiastock.biz/Corporate-Infomation.aspx?securityCode={}".format(stock_code)
    response = requests.get(url, headers={'User-Agent':'test'})
    soup = BeautifulSoup(response.content, "html.parser")
    table = (soup.find_all("label", attrs={"id":'MainContent_lbFinancialInfo_Capital'}))
    return table[0].getText()


def get_stock_price_six_month(ticker):   
    data = yf.download(ticker, period = "6mo", group_by="Ticker", interval='1mo')
    data.dropna(inplace=True)       
    return data


def stock_return_monthly(data):
    try: 
        monthly_return = round((data.iloc[-1,3]-data.iloc[-3,0])/data.iloc[-3,0],4)  
        
    except:
        monthly_return = round((data.iloc[-1,3]-data.iloc[-2,0])/data.iloc[-2,0],4) 
   
    return monthly_return*100

In [7]:
construction_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=3')
consumer_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=4')
energy_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=5')
finance_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=9')
healthcare_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=10')
industrial_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=11')
plantation_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=12')
property_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=13')
reits_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=14')
technology_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=17')
telco_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=18')
transport_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=19')
utilities_sector = sector_classification('https://www.malaysiastock.biz/Listed-Companies.aspx?type=S&s1=20')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [8]:
#     market_cap = []
#     for stock_code in chosen_stock_code:
#         each_stock_market_cap = get_market_cap(stock_code)
#         each_stock_market_cap_formatted = re.sub(r'[: ]', '', each_stock_market_cap)
#         market_cap.append(each_stock_market_cap_formatted)
    
#     market_cap_formatted = []
#     for i in market_cap:
#         if re.findall(r'm',i):
#             a = re.sub(r'm', '', i)
#             market_cap_formatted.append(float(a))
#         elif re.findall(r'b',i):
#             b = re.sub(r'b', '', i)
#             market_cap_formatted.append(float(b)*1000)
        
#     monthly_return = []
#     for ticker in chosen_stock_code:
#         try: 
#             data = get_stock_price(ticker+'.KL')
#             monthly_return.append(stock_return_monthly(data))       
#         except:
#             monthly_return.append(0)
#     data = {}
#     data['Stock_name']=chosen_stock_name
#     data['Stock_code']=chosen_stock_code
#     data['Market_cap_in_million']= market_cap_formatted    
#     data['One_month_return'] = monthly_return
    
#     df = pd.DataFrame(data)
#     df_new = df.sort_values(['Market_cap_in_million'],ascending=False).reset_index()
#     top_twenty = df_new.iloc[:20, :]

In [9]:
def feature_1(chosen_sector, sector_name):
    chosen_stock_code = code_or_name(chosen_sector, 'code')
    chosen_stock_name = code_or_name(chosen_sector, 'name')
    if "5235SS" in chosen_stock_name:
        chosen_stock_name.remove("5235SS")
        
    monthly_return = []
    for ticker in chosen_stock_code:
        try: 
            data = get_stock_price_six_month(ticker+'.KL')
            monthly_return.append(stock_return_monthly(data))       
        except:
            monthly_return.append(0)
    
    average_monthly_return_per_sector = sum(monthly_return)/ len(monthly_return)
    if average_monthly_return_per_sector > 0: 
        return f"{sector_name}: {round(average_monthly_return_per_sector,2)}% \U0001F4C8\n"
    else:
        return f"{sector_name}: {round(average_monthly_return_per_sector,2)}% \U0001F4C9\n"


In [10]:
construction_index = feature_1(construction_sector, 'Construction sector')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

...

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [11]:
energy_index = feature_1(energy_sector, 'Energy sector')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed


...

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [12]:
consumer_index = feature_1(consumer_sector, 'Consumer sector')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


...

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [13]:
finance_index = feature_1(finance_sector, 'Finance sector')

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(
C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


In [14]:
healthcare_index = feature_1(healthcare_sector, 'Healthcare sector')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


In [15]:
industrial_index = feature_1(industrial_sector, 'Industrial sector')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


In [16]:
plantation_index = feature_1(plantation_sector, 'Plantation sector')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


In [17]:
property_index = feature_1(property_sector, 'Property sector')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


In [18]:
reits_index = feature_1(reits_sector, 'Reits sector')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


In [19]:
technology_index = feature_1(technology_sector, 'Technology sector')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


In [20]:
telco_index = feature_1(telco_sector, 'Telco sector')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


In [21]:
transport_index = feature_1(transport_sector, 'Transport sector')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


In [22]:
utilities_index = feature_1(utilities_sector, 'Utilities sector')

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


### Analyse Stock 

In [23]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ta.momentum import RSIIndicator

In [24]:
# Window shifting method to draw support 
# Take a window of 9 candles and find the max of the candles within the window
#Then shift the window up a candle and find the max of the calndlw within the window again.
#Do the same until you shifted 5 times
#If the candle with the highest value remains the same throught the steps, we have a pivot point
def get_stock_price_daily(symbol):
    df_daily = yf.download(symbol, start='2021-06-01')
    return df_daily


def add_ema_and_rsi(df):
    df['EMA25']= df['Close'].ewm(span=25).mean()
    df['EMA50']= df['Close'].ewm(span=50).mean()
    df["RSI"] = RSIIndicator(df['Close'], 14).rsi()
    return df


def get_stock_price_weekly(symbol):
    df_weekly = yf.download(symbol, start='2021-06-01', interval='1wk')
    return df_weekly


def is_far_from_level(value, levels, df):    
    ave =  np.mean(df['High'] - df['Low'])    
    return np.sum([abs(value-level)<ave for _,level in levels])==0


def detect_support_resistance(df):
    levels = []
    max_list = []
    min_list = []
    
    for i in range(5, len(df)-5):
        high_range = df['High'][i-5:i+4]
        current_max = high_range.max()
        if current_max not in max_list:
            max_list = []
        max_list.append(current_max)
        
        if len(max_list) == 5 and is_far_from_level(current_max, levels, df):
            levels.append((high_range.idxmax(), current_max))
      
        low_range = df['Low'][i-5:i+5]
        current_min = low_range.min()
        if current_min not in min_list:
            min_list = []
        min_list.append(current_min)
        
        if len(min_list) == 5 and is_far_from_level(current_min, levels, df):
            levels.append((low_range.idxmin(), current_min))
    return levels

In [25]:
def plot_stock(stock_df_daily, stock_df_weekly):
    dt_all = pd.date_range(start=stock_df_daily.index[0], end=stock_df_daily.index[-1])
    dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(stock_df_daily.index)]
    dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
    
    fig = go.Figure()
    fig.add_trace(go.Candlestick(x=stock_df_daily.index,
                             open=stock_df_daily['Open'],
                             high=stock_df_daily['High'],
                             low=stock_df_daily['Low'],
                             close=stock_df_daily['Close'], 
                             showlegend=False))

    fig.add_trace(go.Scatter(x=stock_df_daily.index, 
                             y=stock_df_daily['EMA25'], 
                             opacity=0.7, 
                             line=dict(color='blue', width=2), 
                             name='EMA 25'))

    fig.add_trace(go.Scatter(x=stock_df_daily.index, 
                             y=stock_df_daily['EMA50'], 
                             opacity=0.7, 
                             line=dict(color='orange', width=2), 
                             name='EMA 50'))

    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])
    fig.update_layout(xaxis_rangeslider_visible=False)
    
    level_weekly = detect_support_resistance(stock_df_weekly)
    for level in level_weekly:
        fig.add_hline(y=level[1],line_dash="dot", opacity=0.7)
    
    return fig.write_image("images/fig1.png")

In [26]:
def analyse_stock(stock_code): 
    try:
        stock_daily = add_ema_and_rsi(get_stock_price_daily(stock_code +'.KL'))
        stock_weekly = add_ema_and_rsi(get_stock_price_daily(stock_code +'.KL'))
        closing_price = round(stock_daily.iloc[-1,3],3)
        candle = stock_daily.iloc[-1]
        candlestick = (candle['Close'] - candle['Open']) / candle['Open']
        candlestick_on_ema = (candle['Open']- candle['EMA25']) / candle['Open']

        if candle['EMA25'] > candle['EMA50']:
            if candlestick < 0 and candle['Close'] < candle['EMA25']:
                return f"The current price of this stock is {closing_price}. The closing price is less than the opening price, lead to a red candlestick. This is not a signal for us to buy in as there is a higher chance the stock will keep going down. We only consider a green candlestick as our signal to buy in! Plus, it seems like this stock has break our EMA 25 support!!! I would highly recommend you to sell it and cut loss. If you bound to hold, please remember the last support is our 8% rules.\U0001F621\n\nBuy: Highly Not Recommended \U0001F44A\nSell: Highly Recommended \U0001F4AA\n"
            elif candlestick < 0 and candle['Close'] > candle['EMA25']:
                return f"The current price of this stock is {closing_price}. The closing price is less than the opening price, lead to a red candlestick. This is not a signal for us to buy in as there is a higher chance the stock will keep going down. We only consider a green candlestick as our signal to buy in! However, if you holding it and it is still up to trend and did not break EMA 25 support, you might consider to hold it a little bit more as maybe is just a retrace. \U0001F44D\n\nBuy: Highly Not Recommended \U0001F44A\nSell: Neutral \U0001F932\nStoploss: {round(candle['EMA25'],3)}"
            else: 
                if candle['Volume'] > 200000:
                    if candle['RSI'] > 80:
                        return f"The current price of this stock is {closing_price}. This stock is having a huge uptrend momentum! However, the Relative Strength Index (RSI) of this stock is too high. \U0001F928 I strongly disagree you to buy this stock. \U0001F645 But, if you holding the stock, you may consider to sell it and take some profit! \U0001F911\n\nBuy: Highly Not Recommended \U0001F44A\nSell: Highly Recommended \U0001F4AA"
                    elif candle['RSI'] > 70:
                        return f"The current price of this stock is {closing_price}. This stock has been consistenly moving up. The Relative Strength Index (RSI) is high. I would not recommend you to buy this stock although it has uptrend potential. There are still 900 stock out there for you to choose. However, if you holding it, you could hold a little bit longer for more profit, or sell half of it to take little bit profit \U0001F601\n\nBuy: Not Recommended \U0001F44A\nSell: Recommended \U0001F4AA"
                    elif candle['RSI'] < 70:
                        if candlestick <= 0.06 and candlestick_on_ema <= 0.05:
                            return f"The current price of this stock is {closing_price}. This stock is uptrending with low Relative Strength Index. Plus, the candlestick is small and close with EMA 25. I would strongly recommend you to buy this stock! \U0001F60D You may set your support at EMA 25 or not losing more than 8% of your buying price. Once it break your support, remember to CUT LOSS! \U0001F976 If you are holding this stock, keep HODL it and wait for the profit! \U0001F911\n\nBuy: Highly Recommended \U0001F4AA \nSell: Highly Not Recommended \U0001F44A\nStoploss: {round(candle['EMA25'],3)} or {round((0.92*closing_price),3)}"
                        elif candlestick > 0.06 and candlestick_on_ema <= 0.05:
                            return f"The current price of this stock is {closing_price}. This stock is uptrending with low Relative Strength Index. However, the candlestick of this stock is too big although it is close with EMA 25. I would not recommend you to buy it because it might retrace at tomorrow. \U0001F914 But if you insist to buy it, set your support at the middle of the candlestick. \U0001F928 If you are holding this stock, you could keep holding it for more profit or sell it if achieve your targetted profit. \U0001F911\n\nBuy: Not Recommended \U0001F44A \nSell: Neutral \U0001F932\nStoploss: {round((candle['Close'] + candle['Open'])/2,3)}"
                        elif candlestick <= 0.06 and candlestick_on_ema > 0.05:
                            return f"The current price of this stock is {closing_price}. This stock is uptrending with low Relative Strength Index. The candlestick is small however, it is too far from EMA 25. I would not recommend you to buy it. \U0001F645 However, if you holding this stock, If you are holding this stock, you could keep holding it for more profit or sell it if achieve your targetted profit. \U0001F911\n\nBuy: Not Recommended \U0001F44A\nSell: Recommended \U0001F4AA"
                        elif candlestick > 0.06 and candlestick_on_ema > 0.05:
                            return f'The current price of this stock is {closing_price}. This stock is uptrending with low Relative Strength Index. However, it has a big big candlestick and far away from EMA 25. I would strongly not recommend you to buy it. \U0001F645 If you are holding this stock, you could keep holding it for more profit or sell it if achieve your targetted profit. \U0001F911\n\nBuy: Highly Not Recommended\U0001F44A\nSell: Recommended \U0001F4AA'
                elif candle['Volume'] < 200000:
                    return f'The current price of this stock is {closing_price}. This stock is uptrending. But the volume trading of this stock is too low. It might lead to low liquidity of the stock and you will have a hard time to sell it once you buy this stock. Hence, with low liquidity of the volume trade I am strongly disagree you to buy it. \U0001F645\n\nBuy: Highly Not Recommended\U0001F44A\nSell: Highly Recommended \U0001F4AA'

        elif candle['EMA25'] < candle['EMA50']:
            return f"The current price of this stock is {closing_price}. This stock is having downtrend. While a stock is having downtrend, you would no need to see other indicator as we only invest into a uptrend momentum stock. I do not want you to lose your money so just ignore this stock! \U0001F621 However, if happen you are holding this stock, please make sure you cut loss immediately. You do not know how far it will drop. \U0001F97A\n\nBuy: Highly Not Recommended\U0001F44A\nSell: Highly Recommended \U0001F4AA"
    except Exception as e:
        print(e)

### Apply Screener to All Stock in Bursa 

In [27]:
def get_stock_code_list_bursa():
    url = "https://www.malaysiastock.biz/Stock-Screener.aspx"
    response = requests.get(url, headers={'User-Agent':'test'})
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find(id = "MainContent2_tbAllStock")
    stock_list = table.find_all('a')
    return [stock_code.get('href')[-4:] for stock_code in stock_list]

In [28]:
#Condition 1: Overall trend is uptrend 
#Condition 2: RSI must be lower than 70, to prevent overbuy and buy high sell low
#Condition 3: Volume must be higher than 200k to prevent low liquidity 
#Condition 4: Close must higher than Open 
#Condition 5: Closing price must be higher than previous closing price 
#Condition 6: Candle Stick must be on EMA 25 
#Condition 7: Price difference must be less than 6% (Close - Open)
#Condition 8: Price different between opening price and EMA 25 must be less than 4%

def check_condition(df):
    candle1 = df.iloc[-1]
    candle2 = df.iloc[-2]
    
    cond1 = candle1['EMA25'] > candle1['EMA50'] 
    cond2 = candle1['RSI'] <= 70 or candle1['RSI'] <= 70
    cond3 = candle1['Volume'] > 200000
    cond4 = candle1['Close'] > candle1['Open']
    cond5 = candle1['Close'] > candle2['Close']
    cond6 = candle1['Open'] > candle1['EMA50'] and candle1['Close'] > candle1['EMA25']
    cond7 = (candle1['Close'] - candle1['Open']) / candle1['Open'] <= 0.06
    cond8 = (candle1['Open']- candle1['EMA25']) / candle1['Open'] <= 0.04
    
    return cond1 and cond2 and cond3 and cond4 and cond5 and cond6 and cond7 and cond8

In [29]:
screened_list = [] 

stock_list = get_stock_code_list_bursa()

C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [30]:
for stock_code in stock_list:    
    try:     
        
        price_chart_df = add_ema_and_rsi(get_stock_price_daily(stock_code+'.KL'))        

        if check_condition(price_chart_df):
            screened_list.append(stock_code)
            
    except Exception as e:
        print(e)

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


C:\Users\Zac\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 3052.KL: No data found, symbol may be delisted
single positional indexer is out-of-bounds


In [31]:
screened_list

['7086',
 '7609',
 '4758',
 '0041',
 '7205',
 '5101',
 '7229',
 '5819',
 '9326',
 '5152',
 '5026',
 '5087',
 '5025',
 '0099',
 '7180',
 '7246',
 '8532',
 '2054',
 '8702',
 '0193']

### Run Telegram Bot

In [32]:
def start(update, context):
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi!')


def help(update, context):
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')

def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)
    
def receive_message(update, context):
    # Query message against DialogFlow model
    PROJECT_ID='stock-recommendator-otox'
    SESSION_ID='123456789'
    LANGUAGE_CODE='EN'

    user_msg=update.message.text    
    response=detect_intent_texts(PROJECT_ID,SESSION_ID,user_msg,LANGUAGE_CODE)
    intent_info=response.query_result.intent.display_name    
    
    if intent_info =='fallback':
        final_message= "I believe I am not smart enough to understand your question or your task. \U0001F97A I am so sorry, but could you please repeat what you trying to ask me in more specific way? \U0001F64F"
   
    elif intent_info == "welcome":
        final_message = "Welcome to Conversational Stock Bot! \U0001F916 Thanks for using me as your personal assitance in stock recommendation \U0001F60D. How can I help you?"

    elif intent_info == 'goodbye':
        final_message = "Is my pleasure to serve you! \U0001F618 Hope to see you again, just hit me a 'Hi' will do! \U0001F607"
        
    elif intent_info == "overview":         
        final_message = f"Here are the past month return performance of the sector in Bursa.\n\n{construction_index}{energy_index}{consumer_index}{finance_index}{healthcare_index}{industrial_index}{plantation_index}{property_index}{reits_index}{technology_index}{telco_index}{transport_index}{utilities_index}\nAfter having a look on it, you may consider which stock to buy or sell according to the sector! \U0001F929"
    
    elif intent_info == 'stock_analyse':
        stock_code=response.query_result.parameters['stock_code']
        try: 
            stock_daily = add_ema_and_rsi(get_stock_price_daily(stock_code +'.KL'))
            stock_weekly = add_ema_and_rsi(get_stock_price_daily(stock_code +'.KL'))
            plot_stock(stock_daily, stock_weekly)
            context.bot.send_photo(chat_id=update.message.chat_id, photo=open('images/fig1.png', 'rb'))
            os.remove('images/fig1.png') 
            final_message = analyse_stock(stock_code) 
        except:
            final_message = 'Seems like you did not provide any stock code or maybe the stock code is delisted in Bursa Malaysia. Perhaps you can enter another stock code for me to analyse? I would love to help you! \U0001F60D'           
        
    
    elif intent_info == 'apply_screener_all_stock':
        final_message = f"Here are some stock you might consider to take a look! You could use me as your analyser to help you analyse the stock! \U0001F929\n{screened_list}\n\nReasons to choose these stocks:\n1. These stocks are in uptrend movement.\n2. These stocks have relative low risk to enter. Price differences between open and closing price are less than 6%.\n3. These stocks have higher trade volume and low relative strength index."
        
    
    
    context.bot.send_message(chat_id=update.message.chat_id, text=final_message)

In [37]:
def main():
    updater = Updater(API_KEY)
    dp = updater.dispatcher
    # Register commands
    # on different commands - answer in Telegram
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help))

    # Handle non-commands
    dp.add_handler(MessageHandler(Filters.text, receive_message))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

Session path: projects/stock-recommendator-otox/agent/sessions/123456789

Session path: projects/stock-recommendator-otox/agent/sessions/123456789

Session path: projects/stock-recommendator-otox/agent/sessions/123456789

Session path: projects/stock-recommendator-otox/agent/sessions/123456789

